# LangChain: Models, Prompts and Output Parsers

## Introduction
This Jupyter notebook demonstrates `how to use LangChain to interact with OpenAI's API for natural language processing tasks`. The notebook covers direct API calls to OpenAI and using LangChain for prompts, models, and output parsers. It provides examples and detailed instructions for setting up and using these tools effectively.
We will cover the following topics:   
- Direct API calls to OpenAI    
- API calls through LangChain:    
  - Prompts     
  - Models    
  - Output parsers  

### Setup

- Get your [OpenAI API Key](https://platform.openai.com/account/api-keys)
- Ensure you have the required packages installed:
``` py
%pip install python-dotenv
%pip install openai
%pip install pydantic==1.10.8
%pip install rich
```

In [ ]:
# Import necessary modules from rich library that helps to improve the readability of nested dictionary outputs
from rich import print
from rich.pretty import Pretty

### Code Examples

### Setting Up OpenAI API

In [ ]:
# Setting Up OpenAI API
# Import necessary libraries
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Load environment variables from a .env file
_ = load_dotenv(find_dotenv()) 

# Setting Up OpenAI API
openai.api_key = os.environ['OPENAI_API_KEY']

**Note** To handle the deprecation of LLM models, we use the current date to select the appropriate model:

In [ ]:
# Handling Model Deprecation
import datetime

# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo"
else:
    llm_model = "gpt-3.5-turbo-0301"

### Direct API Calls to OpenAI

Let's start with a direct API calls to OpenAI.

- Example 1: Simple API Call

In [ ]:
#import openai

# Example 1: Simple API Call
client = openai.OpenAI()  # Create a client instance

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hello world"}]
)

print(response.choices[0].message.content)


Hello! How are you doing today?

- Example 2: Let's change the prompt

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

# Calling the function
result = get_completion("What is 1+1?")
print(result)


1+1 equals 2.

### Using LangChain

- Model

In [6]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse,\
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [7]:
style = """American English \
in a calm and respectful tone
"""

- Prompt Template

In [8]:
prompt = f"""Translate the text \
that is delimited by triple backticks 
into a style that is {style}.
text: ```{customer_email}```
"""

print(prompt)

Translate the text that is delimited by triple backticks 
into a style that is American English in a calm and respectful tone
.
text: ```
Arrr, I be fuming that me blender lid flew off and splattered me kitchen walls with smoothie! And to make matters 
worse,the warranty don't cover the cost of cleaning up me kitchen. I need yer help right now, matey!
```

In [ ]:
# Send the prompt to the OpenAI API and get the completion response
response = get_completion(prompt)

In [ ]:
# Print response
response

"Ah, I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right now, friend."

## Chat API : LangChain

Let's try how we can do the same using LangChain and having firther the same output.

In [ ]:
#!pip install --upgrade langchain

- Model

In [ ]:
# Using LangChain for Advanced API Interactions
from langchain.chat_models import ChatOpenAI

In [ ]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(temperature=0.0, model="gpt-3.5-turbo")

**Note** To control the randomness and creativity of the generated text by an LLM, use temperature = 0.0.

- Prompt template

In [13]:
template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

In [14]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)


In [15]:
prompt_template.messages[0].prompt

PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n')

In [16]:
prompt_template.messages[0].prompt.input_variables

['style', 'text']

- Formatting Messages

In [17]:
customer_style = """American English \
in a calm and respectful tone
"""

In [18]:
customer_email = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [19]:
customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

In [20]:
print(type(customer_messages))
print(type(customer_messages[0]))

<class 'list'>

<class 'langchain_core.messages.human.HumanMessage'>

In [21]:
print(customer_messages[0])

HumanMessage(
    content="Translate the text that is delimited by triple backticks into a style that is American English in a 
calm and respectful tone\n. text: ```\nArrr, I be fuming that me blender lid flew off and splattered me kitchen 
walls with smoothie! And to make matters worse, the warranty don't cover the cost of cleaning up me kitchen. I need
yer help right now, matey!\n```\n",
    additional_kwargs={},
    response_metadata={}
)

- Calling the LLM

In [ ]:
# Call the LLM to translate to the style of the customer message
customer_response = chat.invoke(customer_messages)

In [24]:
print(customer_response.content)

I'm really frustrated that my blender lid flew off and splattered my kitchen walls with smoothie! And to make 
matters worse, the warranty doesn't cover the cost of cleaning up my kitchen. I could really use your help right 
now, friend.

**Explanation output** As it can be seen, the ouptut is similar to that was creating using the API call. 

- Example 3 Let's change the contents of the prompt template

In [25]:
service_reply = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [26]:
service_style_pirate = """\
a polite tone \
that speaks in English Pirate\
"""

In [27]:
service_messages = prompt_template.format_messages(
    style=service_style_pirate,
    text=service_reply)

print(service_messages[0].content)

Translate the text that is delimited by triple backticks into a style that is a polite tone that speaks in English 
Pirate. text: ```Hey there customer, the warranty does not cover cleaning expenses for your kitchen because it's 
your fault that you misused your blender by forgetting to put the lid on before starting the blender. Tough luck! 
See ya!
```

In [ ]:
# Call the LLM to translate to the style of the customer message
service_response = chat(service_messages)
print(service_response.content)

Ahoy there, valued customer! Regrettably, the warranty be not coverin' the costs o' cleanin' yer galley due to yer 
own negligence. Ye see, 'twas yer own doin' that ye forgot to secure the lid afore startin' the blender. 'Tis a 
tough break, indeed! Fare thee well!

## Output Parsers

Let's start with defining how we would like the LLM output to look like:

- Example 1

In [29]:
{
  "gift": False,
  "delivery_days": 5,
  "price_value": "pretty affordable!"
}

{'gift': False, 'delivery_days': 5, 'price_value': 'pretty affordable!'}

In [ ]:
# A sample customer review describing the features, delivery time, and personal experience with a leaf blower
customer_review = """\
This leaf blower is pretty amazing.  It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [ ]:
# Import the ChatPromptTemplate class from the langchain.prompts module
from langchain.prompts import ChatPromptTemplate

# Create a ChatPromptTemplate instance using the review_template string
prompt_template = ChatPromptTemplate.from_template(review_template)

# Print the prompt template to verify its creation and contents
print(prompt_template)

ChatPromptTemplate(
    input_variables=['text'],
    input_types={},
    partial_variables={},
    messages=[
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(
                input_variables=['text'],
                input_types={},
                partial_variables={},
                template='For the following text, extract the following information:\n\ngift: Was the item 
purchased as a gift for someone else? Answer True if yes, False if not or unknown.\n\ndelivery_days: How many days 
did it take for the product to arrive? If this information is not found, output -1.\n\nprice_value: Extract any 
sentences about the value or price,and output them as a comma separated Python list.\n\nFormat the output as JSON 
with the following keys:\ngift\ndelivery_days\nprice_value\n\ntext: {text}\n'
            ),
            additional_kwargs={}
        )
    ]
)

**Explantion output** The provided output is an HTML formatted representation of a Python object, specifically a `ChatPromptTemplate instance`. This instance is configured with various properties and nested objects. To sum up, the ChatPromptTemplate object is used to define a structured prompt template that can be used in a chatbot or language model interaction. The template is designed to extract specific pieces of information from a given text, such as whether an item was a gift, the delivery time, and the value or price of the item. The extracted information is formatted as a JSON object with predefined keys.  

This setup allows for a consistent and structured way to prompt a language model, ensuring that the necessary information is extracted and formatted correctly for further processing or analysis.

In [ ]:
# Format the customer review text using the prompt template
messages = prompt_template.format_messages(text=customer_review)

# Create an instance of ChatOpenAI with specified temperature and model
chat = ChatOpenAI(temperature=0.0, model=llm_model)

# Send the formatted messages to the chat model and get the response
response = chat(messages)

# Print the content of the response
print(response.content)

{
  "gift": true,
  "delivery_days": 2,
  "price_value": ["It's slightly more expensive than the other leaf blowers out there"]
}

In [33]:
type(response.content)

str

### Parse the LLM output string into a Python dictionary

- Import the required modules, the langchain.output_parsers module and the langchain.output_parsers module, from the ResponseSchema class and StructuredOutputParser class, respectively. The ResponseSchema class is used to define the schema for expected responses from the language model. On the other side, the StructuredOutputParser classis is used to parse the structured output from the language model based on the defined schema 

In [ ]:
# Import the ResponseSchema class from the langchain.output_parsers module
from langchain.output_parsers import ResponseSchema

# Import the StructuredOutputParser class from the langchain.output_parsers module
from langchain.output_parsers import StructuredOutputParser

In [36]:
gift_schema = ResponseSchema(name="gift",
                             description="Was the item purchased\
                             as a gift for someone else? \
                             Answer True if yes,\
                             False if not or unknown.")
delivery_days_schema = ResponseSchema(name="delivery_days",
                                      description="How many days\
                                      did it take for the product\
                                      to arrive? If this \
                                      information is not found,\
                                      output -1.")
price_value_schema = ResponseSchema(name="price_value",
                                    description="Extract any\
                                    sentences about the value or \
                                    price, and output them as a \
                                    comma separated Python list.")

response_schemas = [gift_schema, 
                    delivery_days_schema,
                    price_value_schema]

- Create an instance of StructuredOutputParser using predefined response schemas

In [ ]:
# The from_response_schemas method initializes the parser with the given schema definitions
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

- Get the format instructions from the output parser

In [ ]:
# This method retrieves the instructions on how to format the output based on the defined response schema
format_instructions = output_parser.get_format_instructions()

In [39]:
print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing 
"```json" and "```":

```json
{
        "gift": string  // Was the item purchased                             as a gift for someone else?          
Answer True if yes,                             False if not or unknown.
        "delivery_days": string  // How many days                                      did it take for the product 
to arrive? If this                                       information is not found,                                 
output -1.
        "price_value": string  // Extract any                                    sentences about the value or      
price, and output them as a                                     comma separated Python list.
}
```

- Example 2

In [40]:
review_template_2 = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template_2)

messages = prompt.format_messages(text=customer_review, 
                                format_instructions=format_instructions)

In [41]:
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing.  It has four settings:candle blower, gentle breeze, windy city, and 
tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much 
she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the
leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it 
for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing 
"```json" and "```":

```json
{
        "gift": string  // Was the item purchased                             as a gift for someone else?          
Answer True if yes,                             False if not or unknown.
        "delivery_days": string  // How many days                                      did it take for the product 
to arrive? If this                                       information is not found,                                 
output -1.
        "price_value": string  // Extract any                                    sentences about the value or      
price, and output them as a                                     comma separated Python list.
}
```

In [42]:
response = chat(messages)

In [43]:
print(response.content)

```json
{
        "gift": true,
        "delivery_days": 2,
        "price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth
it for the extra features."]
}
```

In [44]:
output_dict = output_parser.parse(response.content)

In [45]:
output_dict

{'gift': True,
 'delivery_days': 2,
 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}

In [46]:
type(output_dict)

dict

In [47]:
output_dict.get('delivery_days')

2

## Conclusion
In this notebook, we explored 1 how to use LangChain and OpenAI API for natural language processing tasks`. We covered direct API calls, model deprecation handling, and advanced interactions using LangChain. By implementing these techniques, developers can efficiently utilize OpenAI's capabilities for various applications.